In [ ]:
# === Define FactSchema ===
# Usage:
# - FactSchema is the canonical set of fact-layer predicates allowed in your system.
# - Each predicate is defined with a signature: a list of ArgSpec entries.
#
# Design rationale:
# - We separate names and argument signatures so that predicates are stable, hashable, and
#   unambiguous even if names collide across domains (namespace/role).
# - This enables FactView filtering and stable predicate_id references for LLM decoding.
#
# ArgSpec fields:
# - datatype: the data type of the argument, e.g., string/int/float
# - namespace: business domain/module/ontology (optional)
# - role: semantic role of the parameter (optional)
import pprint as pp
from symir.rule_ir import ArgSpec
from symir.rule_ir import PredicateSchema as Fact
from symir.rule_ir import FactSchema as FactsRegistry
# datatype名字也许需要改一下, 并且允许用户使用“City:string”这种格式指定name, 如果不指定则默认名字是”Name“, 不指定类型则默认是string
person_name = ArgSpec("string", namespace="person", role="name")
person_address = ArgSpec("string", namespace="address", role="name")

city_name = ArgSpec("City:string", namespace="geo", role="name")
country_name = ArgSpec("Country:string", namespace="geo", role="name")
country_established_year = ArgSpec("int", namespace="geo", role="established_year")

company_name = ArgSpec("string", namespace="org", role="name")
company_worth = ArgSpec("Worth:float", namespace="org", role="worth")

person = Fact("person", [person_name, person_address], description="A person entity")
city = Fact("city", [city_name, country_name], description="A city entity")
company = Fact("company", [company_name,company_worth], description="A company entity")
country = Fact("country", [country_name, country_established_year], description="A country entity")

from symir.rule_ir import Rel
works = Rel("works_at", sub=person, obj=company, description="person works at company")
lives = Rel("lives_in", sub=person, obj=city, description="person lives in city")
located = Rel("located_in", sub=city, obj=country, description="city located in country")
hq = Rel("company_hq", sub=company, obj=city, description="company HQ city")
are_friends = Rel("are_friends", sub=person, obj=person, description="person knows person")

schema = FactsRegistry([person, city, company, country, lives, works, located, hq, are_friends])
print("SCHEMA:")
pp.pprint(schema.to_dict())


In [ ]:
# === 3) Manual facts (FactInstance) ===
# Usage:
# - FactInstance is a single fact record with a predicate_id and constant terms.
# - You can inject facts from any source, not just CSV.
#
# Design rationale:
# - Data access is abstracted by DataProvider, but manual facts are still a first-class path.
# - This is useful for programmatic data or unit tests.
import pprint as pp

from symir.ir.expr_ir import Const
from symir.fact_store.provider import FactInstance as Instance
from symir.probability import ProbabilityConfig, resolve_probability

alice = Instance(id=person.schema_id, terms=["alice","darmstadt"], prob=0.9),
darmstadt = Instance(id=city.schema_id, terms=["darmstadt","germany"]),

facts_from_user = [
    alice,
    Instance(id=person.schema_id, terms=["bob","frankfurt"], prob=0.8),
    Instance(id=city.schema_id, terms=["seattle","usa"], prob=0.95),
    Instance(id=company.schema_id, terms=["openai",10.0]),
    Instance(id=country.schema_id, terms=["usa",1776]),

    Instance(id=lives.schema_id, ref=[alice,darmstadt]), # Use instance reference as terms to link to previously defined facts.
    Instance(id=works.schema_id, subterms=["bob","frankfurt"], objterms=["openai",10.0]), # If only constant terms are provided, according to the assigned parameters of "works", try to resolve "bob" to person_name and "openai" to company_name.
]

# 我想要有一个resolve_rel()方法, 类似neo4j创建关系时的resolve方法，可以根据提供的subterms和objterms自动匹配schema中的参数并创建FactInstance (笛卡尔积式的)。这个方法会根据schema中定义的参数类型和角色来解析输入的terms，并生成正确的FactInstance。